In [39]:
import pandas as pd
import numpy as np
import re

pd.set_option('display.max_columns', None)

In [40]:
df_lyrics2 = pd.read_csv('files/lyrics.csv')
df_last_album = pd.read_csv('files/lyrics_new_album.csv', index_col=0)


In [41]:
df_genres = pd.read_csv('files/genres_to_songs.csv')

In [42]:
df_last_album['lyrics'] = df_last_album['lyrics'].apply(lambda x : re.sub(r'\[.*?\]', '', x).strip().replace('\n', '  '))


In [43]:
df_last_album['Album'] = 'The Tortured Poets Department'

In [44]:
df_last_album = df_last_album.reindex(columns = ['song', 'Album', 'lyrics'])

In [45]:
df_last_album.columns = ['Song', 'Album', 'Lyrics']

In [46]:
df_lyrics2['Album'].unique()

array(['Red (Taylor’s Version)', 'Lover', '1989 (Deluxe)',
       'Midnights (3am Edition)', 'Taylor Swift', 'Speak Now',
       'Cats: Highlights From the Motion Picture Soundtrack',
       'Fearless (Taylor’s Version)', 'reputation', 'evermore',
       'The Taylor Swift Holiday Collection - EP',
       'Women in Music Pt. III (Expanded Edition)',
       'Two Lanes of Freedom (Accelerated Deluxe)',
       'Midnights (Target Exclusive)',
       'Fifty Shades Darker (Original Motion Picture Soundtrack)',
       'Speak Now (Deluxe)', 'Taylor Swift (Best Buy Exclusive)', nan,
       'How Long Do You Think It’s Gonna Last?',
       'The Hunger Games: Songs from District 12 and Beyond',
       'One Chance (Original Motion Picture Soundtrack)',
       '1989 (Taylor’s Version)', 'Love Drunk',
       'Hannah Montana: The Movie',
       'Carolina (From The Motion Picture “Where The Crawdads Sing”) - Single',
       'Christmas Tree Farm', 'folklore', 'folklore (deluxe version)',
       'evermore

In [47]:
albumes = ['Red (Taylor’s Version)', 
           'Lover',
           '1989 (Deluxe)',
           'Midnights (3am Edition)',
           'Taylor Swift',
           'Speak Now (Deluxe)',
           'Fearless (Taylor’s Version)',
           'reputation',
           'evermore (deluxe version)',
           'folklore (deluxe version)',
           'folklore',
           'evermore',
           'Speak Now',
           '1989 (Taylor’s Version)']

df_lyrics_filtrado = df_lyrics2[df_lyrics2['Album'].isin(albumes)]

In [48]:
diccionario = {}

for track in df_lyrics_filtrado['Song']:
    
    df_especific = df_lyrics_filtrado[df_lyrics_filtrado['Song'] == track]
    cadena = "  ".join(df_especific['Lyric'])
    diccionario[track] = cadena

In [49]:
df_lyrics_clean = df_lyrics_filtrado.drop_duplicates(subset = 'Song')

In [50]:
lista_a_dropear = []

for cancion in df_lyrics_clean['Song']:
    
    if '[From the Vault]' in cancion:
        lista_a_dropear.append(cancion)

In [51]:
df_lyrics_clean = df_lyrics_clean[~df_lyrics_clean['Song'].isin(lista_a_dropear)]

In [52]:
df_lyrics_clean['Lyrics'] = df_lyrics_clean.apply(lambda x : diccionario[x['Song']], axis=1)

In [53]:
df_lyrics_clean.drop(columns = ['Lyric', 'Previous Lyric', 'Next Lyric', 'Multiplicity'], inplace=True)

In [54]:
df_lyrics_clean = df_lyrics_clean.reset_index().drop('index', axis=1)

In [55]:
df_lyrics = pd.concat([df_lyrics_clean, df_last_album], ignore_index=True)

In [56]:
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace(' (Taylor’s Version)', ''))
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace('(Taylor’s Version)', ''))
df_lyrics['Album'] = df_lyrics['Album'].apply(lambda x : x.replace(' (Taylor’s Version)', ''))
df_lyrics['Album'] = df_lyrics['Album'].apply(lambda x : x.replace(' (Deluxe)', ''))
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace(' (Pop Version)', ''))
df_lyrics['Song'] = df_lyrics['Song'].apply(lambda x : x.replace('(Radio Edit)', ''))

In [57]:
diccionario_cambios = {'...Ready for It?' : '…Ready for It?',
                       'So It Goes...' : 'So It Goes…',
                       '’tis the damn season' : '‘tis the damn season',
                       'l​ong story short' : 'long story short',
                       "Who's Afraid of Little Old Me?" : 'Who’s Afraid of Little Old Me?',
                       'You’re Not Sorry' : "You're Not Sorry",
                       'You’re On Your Own, Kid' : "You're on Your Own, Kid",
                       'Would’ve, Could’ve, Should’ve' : "Would've, Could've, Should've"}

df_lyrics['Song'] = df_lyrics['Song'].map(diccionario_cambios).fillna(df_lyrics['Song'])

In [58]:
words = ['In ', 'On ', 'At ', 'By ', 'For ', 'With ', 'From ', 'To ', 'Into ', 'Onto ', 'Of ', 'About ', 'Above ', 'Below ', 'Under ', 'Over ', 'Through ', 'Between ', 'Among ', 'Within ', 'The ', 'But ', 'A ', 'Oh ', 'Oh,', 'It ', 'If ', 'How ', 'Where ', 'When ', 'What ', 'Who ', 'Why ', 'But ', 'Because ', 'An ', 'And ', 'May ']
pattern = r"'[^ ]*"
df_lyrics['Lyrics'] = df_lyrics['Lyrics'].apply(lambda x : re.sub(pattern, '', x))

for word in words:
    df_lyrics['Lyrics'] = df_lyrics['Lyrics'].str.replace(word , ' ')
    df_lyrics['Lyrics'] = df_lyrics['Lyrics'].str.replace(f' {word.lower()}' , ' ')


In [59]:
df_lyrics['song'] = df_lyrics['Song'].str.lower().str.strip()
df_genres['song'] = df_genres['song'].str.lower().str.strip()

In [60]:
df_genres = df_genres.merge(df_lyrics, left_on='song', right_on='song', how='left')

In [61]:
df_genres[df_genres['Song'].isnull()]['song'].unique()

array([], dtype=object)

In [62]:
df_genres.drop(['song', 'album'], axis=1, inplace=True)

In [63]:
df_genres = df_genres.reindex(columns = ['Album', 'Song', 'genre', 'Lyrics'])

In [64]:
df_genres = df_genres.sort_values('Song', ascending=True)

In [65]:
df_genres

,Album,Song,genre,Lyrics
68,Red,22,Fiestas y Tradiciones,feels like perfect night dress up like hips...
57,Red,22,Música disco,feels like perfect night dress up like hips...
74,Red,22,Dance pop,feels like perfect night dress up like hips...
75,Red,22,Bubblegum pop,feels like perfect night dress up like hips...
51,Red,22,Pop,feels like perfect night dress up like hips...
...,...,...,...,...
101,reputation,…Ready for It?,Pop,Knew he was killer first time that I saw him ...
107,reputation,…Ready for It?,Música infantil,Knew he was killer first time that I saw him ...
110,reputation,…Ready for It?,Electropop,Knew he was killer first time that I saw him ...
125,reputation,…Ready for It?,Música industrial,Knew he was killer first time that I saw him ...


In [77]:
#df_genres.to_csv('files/taylor_full_lyrics_w_genre.csv', index=False)

## Limpieza DF Twitter

In [67]:
df_tuits = pd.read_csv('files/TaylorSwift13.csv')

In [68]:
df_tuits.drop('created_at', axis=1, inplace=True)

In [69]:
df_tuits['date'] = df_tuits['date'].apply(lambda x : x.replace('+00:00', ''))

In [70]:
df_tuits['media'] = df_tuits['media'].apply(lambda x : 1 if isinstance(x, str) else 0)
df_tuits['outlinks'] = df_tuits['outlinks'].apply(lambda x : 0 if x == '[]' else 1)
df_tuits['quotedTweet'] = df_tuits['quotedTweet'].apply(lambda x : 1 if isinstance(x, str) else 0)
df_tuits['retweetedTweet'] = df_tuits['retweetedTweet'].apply(lambda x : 1 if isinstance(x, str) else 0)

df_tuits['content'] = df_tuits['content'].apply(lambda x : x.replace('\n', ' '))

In [71]:
df_tuits['users'] = df_tuits.apply(lambda x : re.findall(r'@\S+', x['content']), axis=1)

In [72]:
df_citados = df_tuits[['content', 'date', 'users']]

In [73]:
df_tuits.drop('users', axis=1, inplace=True)

In [74]:
df_citados = df_citados.explode('users')

In [75]:
df_citados.dropna(subset='users', inplace = True)

In [76]:
#df_tuits.to_csv('files/taylor_tuits_limpio.csv', index=False)
#df_citados.to_csv('files/taylor_tuits_citados.csv', index=False)